In [1]:
from env2048.vectenv import Vectorized2048Env
import torch

/Users/marshingjay/miniconda3/envs/python3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
num_boards = 100000
v = Vectorized2048Env(num_boards, torch.device('cpu'))
v.reset()

In [6]:
moves = torch.randint(0, 4, (num_boards,))
for i in range(1000):
    v.step(moves)